In [1]:
import pickle

with open("/kaggle/input/mai-processed/X.pickle", "rb") as p:
    X = pickle.load(p)
with open("/kaggle/input/mai-processed/y.pickle", "rb") as p:
    y = pickle.load(p)

In [2]:
print(len(X))
print(len(y))

824
824


In [3]:
mai = {}
t = []
l = []
for text, label in zip(X, y):
    t.append(text)
    l.append(label)
mai["text"] = t
mai["label"] = l
assert(len(mai["text"]) == len(mai["label"]))

In [4]:
!pip install datasets

     |████████████████████████████████| 325 kB 574 kB/s            
     |████████████████████████████████| 212 kB 9.0 MB/s            


In [5]:
from datasets import Dataset

mai_data = Dataset.from_dict(mai)

In [6]:
import gc

del X
del y
gc.collect()

95

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot_small-90M")

Downloading:   0%|          | 0.00/205 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/941k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/337k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [8]:
def tok(data):
    text = data["text"]
    label = data["label"]
    t_text = tokenizer(text, padding="max_length", truncation=True, return_tensors="tf")
    t_label = tokenizer(label, padding="max_length", truncation=True, return_tensors="tf")
    output = {'input_ids': t_text["input_ids"][0], 'attention_mask': t_text["attention_mask"][0], 'labels': t_label["input_ids"][0], 'decoder_attention_mask': t_label["attention_mask"][0]}
    return output

tokenized_mai = mai_data.map(tok)

  0%|          | 0/824 [00:00<?, ?ex/s]

2022-03-18 01:06:31.405542: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-18 01:06:31.406538: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-18 01:06:31.407173: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-18 01:06:31.409134: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [9]:
import tensorflow as tf

def to_tf_dataset(dataset):  
    columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']
    dataset.set_format(type='tensorflow', columns=columns)
    return_types = {'input_ids':tf.int32, 'attention_mask':tf.int32, 
                'labels':tf.int32, 'decoder_attention_mask':tf.int32,  }
    return_shapes = {'input_ids': tf.TensorShape([None]), 'attention_mask': tf.TensorShape([None]), 
                  'labels': tf.TensorShape([None]), 'decoder_attention_mask':tf.TensorShape([None])}
    ds = tf.data.Dataset.from_generator(lambda : dataset, return_types, return_shapes)
    return ds

tf_mai = to_tf_dataset(tokenized_mai)

In [10]:
tf_mai = tf_mai.batch(8)

In [11]:
import tensorflow_addons as tfa
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot_small-90M")

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5))

Downloading:   0%|          | 0.00/334M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBlenderbotSmallForConditionalGeneration.

All the layers of TFBlenderbotSmallForConditionalGeneration were initialized from the model checkpoint at facebook/blenderbot_small-90M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBlenderbotSmallForConditionalGeneration for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as the 'labels' key of the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [12]:
model.fit(tf_mai, epochs=5)

2022-03-18 01:07:02.024070: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
103/103 [==============================] - 115s 763ms/step - loss: 4.3520
Epoch 2/5
103/103 [==============================] - 79s 763ms/step - loss: 3.7792
Epoch 3/5
103/103 [==============================] - 79s 763ms/step - loss: 3.4798
Epoch 4/5
103/103 [==============================] - 79s 763ms/step - loss: 3.2375
Epoch 5/5
103/103 [==============================] - 79s 765ms/step - loss: 3.0339


In [13]:
model.save_pretrained("/kaggle/working")

In [14]:
input_text = "I have trouble managing my time and it has caused me to perform poorly academically. How can I manage my time better?"
encoded_query = tokenizer(input_text, return_tensors="tf", pad_to_max_length=True, truncation=True)
input_ids = encoded_query["input_ids"]
attention_mask = encoded_query["attention_mask"]
answer = model.generate(input_ids, attention_mask=attention_mask)
decoded_answer = tokenizer.decode(answer.numpy()[0])
print(decoded_answer)

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


__start__ i'm sorry to hear that this has caused you to perform poorly academically. i would encourage you to seek professional help immediately if you are struggling with time management. there are a few things that can help you to manage your time better. first of all, it is important to remember that time management is a skill that can be practiced over and over again. you may be able to learn how to manage time differently from time to time. for example, if you practice for a few hours a week, you may find that you are able to control your time differently than if you practiced for a certain amount of time.


In [15]:
!zip -r file.zip "/kaggle/working"

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/tf_model.h5 (deflated 8%)
  adding: kaggle/working/config.json (deflated 60%)
  adding: kaggle/working/__notebook__.ipynb (deflated 94%)
